# Model Selection and Deploying

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.model_selection import cross_val_score
import warnings

In [2]:
''' 
This class is designed to load, preprocess, and split data into train and test sets 
so that it is ready to be used by models.
'''
class Data:
    def __init__(self, file, cols, target_col, preprocess_labels, preprocess_bins):
        self.cols = list(cols)
        self.target_col = target_col
        self.X_train, self.X_test, self.y_train, self.y_test = self._preprocess_data(file, cols, target_col, preprocess_labels, preprocess_bins)
    
    '''Load the data into a data frame, convert target into binary, and split the data into train and test sets'''
    def _preprocess_data(self, file, cols, target_col, preprocess_labels, preprocess_bins):
        # transform file to dataframe
        file_df = self._load_data(file)
        # rename variables: replace space with underscores (makes it easier to reference)
        file_df.columns = [c.lower().replace(' ', '_') for c in file_df.columns]
        # converting target variables to corresponding labels
        file_df[target_col] = pd.cut(file_df[target_col], bins = preprocess_bins, labels = preprocess_labels)
        
        return self._split_data_train_test(X=file_df[cols], y=file_df[target_col])
      
    '''Load the csv with pandas.'''
    def _load_data(self, file):
        return pd.read_csv(file, sep=';')
    
    ''' Perform a 60% training and 40% testing split'''
    def _split_data_train_test(self, X, y):
        # 60% training, 40% testing
        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.4, random_state=10)
        # optimize performance of train set
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.fit_transform(X_test)
        
        return X_train, X_test, y_train, y_test
    

In [3]:
'''
This class is designed to hold all of our models and be able to fit and predict with them. 
Also, it used to perform cross validation.
'''
class ModelContainer:
    def __init__(self, models=[]):
        self.models = models
        self.best_model = None
        self.predictions = None
        self.mean_precision = {}
    
    '''Add a model to the container. '''    
    def add_model(self, model):
        self.models.append(model)
        
    '''Perform k-fold cross validation.'''   
    def cross_validate(self, data, k):
        for model in self.models:
            score = np.mean(cross_val_score(model, data.X_test, data.y_test, scoring = 'precision', cv=k))
            self.mean_precision[model] = score
    
    '''Selects model with highest preicision.'''    
    def select_best_model(self):
        self.best_model = max(self.mean_precision, key=self.mean_precision.get)
    
    '''Fits best model.'''
    def best_model_fit(self, features, targets):
        self.best_model.fit(features, targets)
    
    '''Scores features using best model.'''
    def best_model_predict(self, features):
        self.predictions = self.best_model.predict(features)
    
    '''Prints summary of models and the best model.'''
    def print_results(self):
        print('Model Summaries:')
        for model in models.mean_precision:
            print('\n', model, '- Precision:', models.mean_precision[model])
        print('\nBest Model:\n', models.best_model)
        print('\nPrecision of Best Model\n', models.mean_precision[models.best_model])

## Model Selection (using cross validation)

We will now evaluate our models using 10-fold cross-validation. The model with the best cross-validation score will be the model we select.

In [4]:
# ignore warnings
warnings.filterwarnings(action = 'ignore')

In [5]:
# define parameters
file = 'winequality-red.csv'
cols = ['fixed_acidity', 'volatile_acidity','citric_acid',
        'residual_sugar','chlorides','free_sulfur_dioxide',
        'total_sulfur_dioxide','density','ph',
        'sulphates','alcohol']
target_col = 'quality'
preprocess_labels = [0,1]
preprocess_bins = (2, 6, 8)

In [6]:
# create a data object
data = Data(file, cols, target_col, preprocess_labels, preprocess_bins)

In [7]:
# add models to ModelContainer
models = ModelContainer()
models.add_model(RandomForestClassifier(max_depth = 12,
                            min_samples_split = 6,
                            n_estimators = 104,
                            max_features = 'auto',
                            min_samples_leaf = 30,
                            random_state = 10))
models.add_model(LogisticRegression(C = 0.01, 
                                    penalty = 'l2', 
                                    class_weight = None, 
                                    random_state = 10))
models.add_model(SVC(kernel = 'rbf',
                     gamma = 1,
                     C = 1,
                     random_state = 10))

In [8]:
# perform cross validation and show results
models.cross_validate(data, k=10)
models.select_best_model()
models.print_results()

Model Summaries:

 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=12, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=30, min_samples_split=6,
            min_weight_fraction_leaf=0.0, n_estimators=104, n_jobs=None,
            oob_score=False, random_state=10, verbose=0, warm_start=False) - Precision: 0.3

 LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=10, solver='warn',
          tol=0.0001, verbose=0, warm_start=False) - Precision: 0.589047619047619

 SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=10, shrinking=True,
  tol=0.001, verbose=False) - Precision: 0.8333333333333333

Best Mo

We see that Support Vector Machines has the highest score! This comes as no suprise, as when we were doing our hyperparameter tuning, Support Vector Machines was able to define a boundary that most clearly incapsulates the relationship between 'good' and 'bad' wines.

## Deploying the Final Model

In [9]:
# train the final model with hand-selected parameters
models.best_model_fit(data.X_train, data.y_train)

In [10]:
import pickle

# save the classifier
with open('fitted-svm.pkl', 'wb') as fid:
    pickle.dump(models.best_model, fid)  
    
# load predictions to csv
svm_final_predictions = pd.DataFrame(models.best_model_predict(data.X_test), columns=['predictions']).to_csv('red-wine-svm-final-prediction.csv')


# Conclusion
We entered this project looking at specific characteristics that makeup a "good" wine and found the most prominent correlation to be alcohol content, making it the top factor to judge a bottle of wine. However, after analysis of other components such as sulphates, citric acid and pH level, we also found postive correlations with these features and the overall quality of a wine. At first glance, these features were overlooked due to their low amounts. The combination of these features working together really helps to enhance the quality of a wine.

What appears internally in the finished product of a wine does not completely tell the whole story of a wine's quality. Other measures, often intangible, can also directly impact a bottle of wine's quality. These factors incldue the grape varietials' environemntal conditions and the equipment used in producing the wine.

From our machine learning analysis, the best classifier to use is the Support Vector Machines since it is the best way to differentiate qualities through a non-linear decision boundary, leading to more accurate findings. This helps support my innitial feature exploration conclusion that "good" quality wines may possess very similar characteristics to "bad" quality wines. It would be interesting to collect more date to re-evaluate this dataset, especially with wines scoring in the 9s and 10s or below a score of 5. I believe this will definitely change the analysis. 

While what constitutes as a "good" quality wine can be precisely determined by tests like this, at the end of the day, the preferences over bottles of wine rely heavily on personal taste and past experiences. Our palettes' understanding and inclination towards a bottle of wine is created through going through all the archives of what we have previously tasted before. So a bottle of wine can taste like apple juice to one person and kiwis to another. Therefore, the factors outlined through this project serve as beginning standards of a good quality wine to help consumers make better economic decisions when it comes to wine. As for taste, I suggest to get drinking and discovering your favorite bottle!

## Ways to improve
1. Explore more machine learning algorithms that are able to detect non-linear relationships.
2. Grid Search for optimal parameters.
3. Gather more samples and see if we can evaluate on a different metric or possibly perform multi-class classification (if the classes are balanced). 